In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode

from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
from syft.core.frameworks.torch.tensor import _MPCTensor
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

In [2]:
x = torch.LongTensor([[1,2],[3,4]])
y = torch.LongTensor([[5,6],[7,8]])

x = x.share(bob, alice)
y = y.share(bob, alice)

In [7]:
z = -x
assert (z.get() == torch.LongTensor([[-1,-2], [-3, -4]])).all()

z = x - y
assert (z.get() == torch.LongTensor([[-4,-4],[-4,-4]])).all()

assert (x.mm(y).get() - torch.LongTensor([[18,22],[43,49]])).abs().sum() < 5

In [34]:
x = torch.LongTensor([[1,-2],[3,-4]])
y = torch.LongTensor([[5,6],[7,8]])

target = x.mm(y)

x = x.share(bob, alice)
y = y.share(bob, alice)

result = x.mm(y)
(result.get() - target).abs().sum()<5

In [36]:
target


 -9 -10
-13 -14
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

True

In [16]:
z.get()


 4  4
 4  4
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]